In [3]:
import os
import numpy as np
import pandas as pd
from time import time
from tqdm import tqdm
import json
import chromadb
from chromadb.config import Settings

from src.datasets.caltech256 import Caltech256DataModule
from src.featuring.rgb_histogram import RGBHistogram
from src.metrics import average_precision, recall, hit_rate

# Tạo thư mục output
os.makedirs('out', exist_ok=True)

print("✅ Imports hoàn tất!")


✅ Imports hoàn tất!


In [6]:
# ======================== CONFIGURATIONS ========================
CONFIGS = [
    # (n_bin, h_type, metric)
    (4, 'global', 'cosine'),
    (4, 'global', 'l2'),
    (4, 'region', 'cosine'), 
    (4, 'region', 'l2'),
    (8, 'global', 'cosine'),
    (8, 'global', 'l2'),
    (8, 'region', 'cosine'),
    (8, 'region', 'l2'),
]

# Cấu hình evaluation
TEST_SIZE = 5956     # Số lượng test samples để evaluation
N_SLICE = 3          # Số slice cho region histogram
K_VALUES = [1, 5, 10]  # Top-k cho evaluation

# ChromaDB settings
CHROMA_DIR = "chroma_storage"
COLLECTION_PREFIX = "rgb_hist"

print(f"📋 Tổng cấu hình để test: {len(CONFIGS)}")
print(f"📊 Test size: {TEST_SIZE}")
print(f"💾 ChromaDB storage: {CHROMA_DIR}")
print(f"🎯 Evaluation k-values: {K_VALUES}")


📋 Tổng cấu hình để test: 8
📊 Test size: 5956
💾 ChromaDB storage: chroma_storage
🎯 Evaluation k-values: [1, 5, 10]


In [7]:
# Setup dataset
dataset_root = os.path.abspath('data/caltech-256/256_ObjectCategories')
print(f"📂 Dataset path: {dataset_root}")
print(f"📂 Exists: {os.path.exists(dataset_root)}")

if os.path.exists(dataset_root):
    data_module = Caltech256DataModule(batch_size=32, root=dataset_root)
    data_module.setup()
    train_loader = data_module.train_dataloader()
    test_loader = data_module.test_dataloader()
    
    train_size = len(train_loader.dataset)
    test_size = len(test_loader.dataset)
    total_size = train_size + test_size
    
    print(f"✅ Dataset loaded thành công!")
    print(f"📊 Training: {train_size} samples ({train_size/total_size*100:.1f}%)")
    print(f"📊 Testing: {test_size} samples ({test_size/total_size*100:.1f}%)")
    print(f"📊 Total: {total_size} samples")
else:
    print("❌ Dataset không tìm thấy!")
    print("Vui lòng đảm bảo dataset ở đường dẫn đúng.")


📂 Dataset path: D:\AI\Food-CBIR\src\evaluation\data\caltech-256\256_ObjectCategories
📂 Exists: True
📂 Found 256 valid categories
📊 Loaded 29780 total images from 256 classes
📋 Train: 23824 images
📂 Found 256 valid categories
📊 Loaded 29780 total images from 256 classes
📋 Test: 5956 images
Train: 23824, Test: 5956
✅ Dataset loaded thành công!
📊 Training: 23824 samples (80.0%)
📊 Testing: 5956 samples (20.0%)
📊 Total: 29780 samples


In [11]:
# Setup GPU/CPU device
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Device: {device}")

if device.type == "cuda":
    print(f"📱 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"🔥 CUDA Version: {torch.version.cuda}")
    # Clear any existing GPU cache
    torch.cuda.empty_cache()
else:
    print("💻 Using CPU (GPU not available)")


🚀 Device: cuda
📱 GPU: NVIDIA GeForce RTX 4060 Laptop GPU
💾 GPU Memory: 8.0 GB
🔥 CUDA Version: 12.1


In [12]:
# Setup ChromaDB
os.makedirs(CHROMA_DIR, exist_ok=True)

chroma_client = chromadb.PersistentClient(
    path=CHROMA_DIR,
    settings=Settings(
        anonymized_telemetry=False,
        allow_reset=True
    )
)

print(f"✅ ChromaDB initialized tại: {CHROMA_DIR}")
print(f"📊 Existing collections: {len(chroma_client.list_collections())}")


✅ ChromaDB initialized tại: chroma_storage
📊 Existing collections: 1


In [13]:
def get_collection_name(n_bin, h_type, metric):
    """Generate collection name for configuration"""
    return f"{COLLECTION_PREFIX}_{n_bin}bin_{h_type}_{metric}"

def extract_and_store_features(n_bin, h_type, metric, force_reindex=False):
    """Extract features và store vào ChromaDB"""
    collection_name = get_collection_name(n_bin, h_type, metric)
    
    # Kiểm tra xem collection đã tồn tại chưa
    try:
        collection = chroma_client.get_collection(collection_name)
        count = collection.count()
        if count > 0 and not force_reindex:
            print(f"📂 Sử dụng collection có sẵn: {collection_name} ({count} vectors)")
            return collection, count
    except:
        pass  # Collection chưa tồn tại
    
    print(f"🔧 Tạo collection mới: {collection_name}")
    print(f"   Cấu hình: n_bin={n_bin}, h_type={h_type}, metric={metric}")
    
    # Xóa collection cũ nếu có
    try:
        chroma_client.delete_collection(collection_name)
    except:
        pass
    
    # Tạo collection mới
    chroma_metric = "cosine" if metric == "cosine" else "l2"
    collection = chroma_client.create_collection(
        name=collection_name,
        metadata={"n_bin": n_bin, "h_type": h_type, "metric": metric},
        embedding_function=None
    )
    
    # Extract features
    feature_extractor = RGBHistogram(n_bin=n_bin, h_type=h_type, n_slice=N_SLICE)
    
    print("📊 Extracting features từ training set...")
    stored_count = 0
    train_labels_map = {}
    
    start_time = time()
    
    for batch_idx, (images, labels, _) in enumerate(tqdm(train_loader, desc="Indexing")):
        # Move to GPU if available
        if device.type == "cuda":
            images = images.to(device)
        
        # Convert to numpy
        images_np = (images.cpu().numpy().transpose(0, 2, 3, 1) * 255).astype(np.uint8)
        
        # Extract features cho batch
        batch_features = []
        batch_ids = []
        batch_metadatas = []
        
        for i, (img, label) in enumerate(zip(images_np, labels)):
            feature = feature_extractor(img)
            
            vector_id = f"train_{stored_count + i}"
            
            batch_features.append(feature.tolist())
            batch_ids.append(vector_id)
            batch_metadatas.append({
                "type": "train",
                "label": int(label),
                "index": stored_count + i
            })
            
            train_labels_map[vector_id] = int(label)
        
        # Lưu batch vào ChromaDB
        if batch_features:
            collection.add(
                embeddings=batch_features,
                ids=batch_ids,
                metadatas=batch_metadatas
            )
        
        stored_count += len(batch_features)
    
    indexing_time = time() - start_time
    
    # Lưu labels mapping
    try:
        labels_json = json.dumps(train_labels_map)
        collection.add(
            embeddings=[[0.0] * len(batch_features[0])],
            ids=["_labels_map_"],
            metadatas=[{"type": "labels_map"}],
            documents=[labels_json]
        )
    except Exception as e:
        print(f"⚠️  Không thể lưu labels mapping: {e}")
    
    print(f"✅ Hoàn thành indexing: {stored_count} vectors trong {indexing_time:.2f}s")
    print(f"⚡ Tốc độ: {stored_count/indexing_time:.1f} vectors/s")
    
    return collection, stored_count

print("📋 Hàm indexing đã sẵn sàng!")


📋 Hàm indexing đã sẵn sàng!


In [14]:
# CHẠY INDEXING CHO TẤT CẢ CẤU HÌNH
print(f"🚀 Bắt đầu indexing cho {len(CONFIGS)} cấu hình...")
print(f"📊 Sẽ index {train_size} training samples cho mỗi cấu hình")
print("⏳ Quá trình này có thể mất vài phút...")

indexing_results = []

for i, (n_bin, h_type, metric) in enumerate(CONFIGS, 1):
    print(f"\n{'='*60}")
    print(f"Indexing {i}/{len(CONFIGS)}: {n_bin}bin_{h_type}_{metric}")
    print(f"{'='*60}")
    
    try:
        start_time = time()
        collection, count = extract_and_store_features(n_bin, h_type, metric)
        end_time = time()
        
        result = {
            'config': f"{n_bin}bin_{h_type}_{metric}",
            'n_bin': n_bin,
            'h_type': h_type,
            'metric': metric,
            'vectors_count': count,
            'indexing_time': end_time - start_time
        }
        indexing_results.append(result)
        
        print(f"✅ Hoàn thành: {count} vectors")
        
    except Exception as e:
        print(f"❌ Lỗi khi indexing {n_bin}bin_{h_type}_{metric}: {e}")
        import traceback
        traceback.print_exc()

print(f"\n🎉 Hoàn thành indexing cho tất cả {len(indexing_results)} cấu hình!")


🚀 Bắt đầu indexing cho 8 cấu hình...
📊 Sẽ index 23824 training samples cho mỗi cấu hình
⏳ Quá trình này có thể mất vài phút...

Indexing 1/8: 4bin_global_cosine
📂 Sử dụng collection có sẵn: rgb_hist_4bin_global_cosine (1632 vectors)
✅ Hoàn thành: 1632 vectors

Indexing 2/8: 4bin_global_l2
🔧 Tạo collection mới: rgb_hist_4bin_global_l2
   Cấu hình: n_bin=4, h_type=global, metric=l2
📊 Extracting features từ training set...


Indexing: 100%|██████████| 745/745 [04:50<00:00,  2.57it/s]


✅ Hoàn thành indexing: 23824 vectors trong 290.03s
⚡ Tốc độ: 82.1 vectors/s
✅ Hoàn thành: 23824 vectors

Indexing 3/8: 4bin_region_cosine
🔧 Tạo collection mới: rgb_hist_4bin_region_cosine
   Cấu hình: n_bin=4, h_type=region, metric=cosine
📊 Extracting features từ training set...


Indexing: 100%|██████████| 745/745 [06:18<00:00,  1.97it/s]


✅ Hoàn thành indexing: 23824 vectors trong 378.93s
⚡ Tốc độ: 62.9 vectors/s
✅ Hoàn thành: 23824 vectors

Indexing 4/8: 4bin_region_l2
🔧 Tạo collection mới: rgb_hist_4bin_region_l2
   Cấu hình: n_bin=4, h_type=region, metric=l2
📊 Extracting features từ training set...


...

✅ Hoàn thành indexing: 23824 vectors trong 185.58s
⚡ Tốc độ: 128.4 vectors/s
✅ Hoàn thành: 23824 vectors

Indexing 7/8: 8bin_region_cosine
🔧 Tạo collection mới: rgb_hist_8bin_region_cosine
   Cấu hình: n_bin=8, h_type=region, metric=cosine
📊 Extracting features từ training set...


Indexing: 100%|██████████| 745/745 [05:30<00:00,  2.25it/s]


✅ Hoàn thành indexing: 23824 vectors trong 330.44s
⚡ Tốc độ: 72.1 vectors/s
✅ Hoàn thành: 23824 vectors

Indexing 8/8: 8bin_region_l2
🔧 Tạo collection mới: rgb_hist_8bin_region_l2
   Cấu hình: n_bin=8, h_type=region, metric=l2
📊 Extracting features từ training set...


Indexing: 100%|██████████| 745/745 [05:33<00:00,  2.24it/s]

✅ Hoàn thành indexing: 23824 vectors trong 333.03s
⚡ Tốc độ: 71.5 vectors/s
✅ Hoàn thành: 23824 vectors

🎉 Hoàn thành indexing cho tất cả 8 cấu hình!


In [15]:
# HIỂN THỊ KẾT QUẢ INDEXING
if indexing_results:
    print("\n📊 INDEXING SUMMARY:")
    print("-" * 60)
    for result in indexing_results:
        print(f"{result['config']:<20} | {result['vectors_count']:>6} vectors | {result['indexing_time']:>6.1f}s")
    
    # Lưu kết quả
    indexing_df = pd.DataFrame(indexing_results)
    indexing_df.to_csv('out/indexing_results.csv', index=False)
    print(f"\n💾 Kết quả indexing lưu tại: out/indexing_results.csv")
    
    print(f"\n📊 Tổng kết:")
    print(f"   - Tổng collections: {len(chroma_client.list_collections())}")
    print(f"   - Tổng vectors: {sum(r['vectors_count'] for r in indexing_results):,}")
    print(f"   - Tổng thời gian: {sum(r['indexing_time'] for r in indexing_results):.1f}s")
else:
    print("Không có kết quả indexing nào!")

# GPU memory cleanup after indexing
if device.type == "cuda":
    torch.cuda.empty_cache()
    print("🧹 GPU memory cleaned up after indexing")



📊 INDEXING SUMMARY:
------------------------------------------------------------
4bin_global_cosine   |   1632 vectors |    0.0s
4bin_global_l2       |  23824 vectors |  290.1s
4bin_region_cosine   |  23824 vectors |  379.1s
4bin_region_l2       |  23824 vectors |  378.0s
8bin_global_cosine   |  23824 vectors |  327.4s
8bin_global_l2       |  23824 vectors |  185.7s
8bin_region_cosine   |  23824 vectors |  330.6s
8bin_region_l2       |  23824 vectors |  333.2s

💾 Kết quả indexing lưu tại: out/indexing_results.csv

📊 Tổng kết:
   - Tổng collections: 8
   - Tổng vectors: 168,400
   - Tổng thời gian: 2224.2s
🧹 GPU memory cleaned up after indexing


In [16]:
def get_train_labels(collection):
    """Lấy labels mapping từ collection"""
    try:
        # Thử lấy từ labels map
        result = collection.get(ids=["_labels_map_"])
        if result['documents'] and len(result['documents']) > 0:
            return json.loads(result['documents'][0])
    except:
        pass
    
    # Fallback: lấy từ metadata
    all_data = collection.get(where={"type": "train"})
    labels_map = {}
    for id, metadata in zip(all_data['ids'], all_data['metadatas']):
        if 'label' in metadata:
            labels_map[id] = metadata['label']
    
    return labels_map

def evaluate_configuration(n_bin, h_type, metric):
    """Evaluate một cấu hình"""
    collection_name = get_collection_name(n_bin, h_type, metric)
    
    try:
        collection = chroma_client.get_collection(collection_name)
        train_labels_map = get_train_labels(collection)
        
        print(f"📊 Found {len(train_labels_map)} training vectors")
        
        if len(train_labels_map) == 0:
            print("❌ Không có training data! Vui lòng chạy indexing trước.")
            return None
        
    except Exception as e:
        print(f"❌ Không tìm thấy collection {collection_name}: {e}")
        print("Vui lòng chạy indexing trước!")
        return None
    
    # Extract test features và evaluate
    feature_extractor = RGBHistogram(n_bin=n_bin, h_type=h_type, n_slice=N_SLICE)
    
    all_results = []
    test_labels = []
    tested_count = 0
    
    start_time = time()
    
    for images, labels, _ in tqdm(test_loader, desc="Testing"):
        if tested_count >= TEST_SIZE:
            break
        
        count = min(len(images), TEST_SIZE - tested_count)
        batch_images = images[:count]
        batch_labels = labels[:count]
        
        # Move to GPU if available
        if device.type == "cuda":
            batch_images = batch_images.to(device)
        
        # Convert to numpy
        images_np = (batch_images.cpu().numpy().transpose(0, 2, 3, 1) * 255).astype(np.uint8)
        
        for img, label in zip(images_np, batch_labels):
            if tested_count >= TEST_SIZE:
                break
            
            # Extract test feature
            test_feature = feature_extractor(img)
            
            # Query ChromaDB
            results = collection.query(
                query_embeddings=[test_feature.tolist()],
                n_results=max(K_VALUES),
                where={"type": "train"}
            )
            
            # Lấy labels của kết quả
            if results['ids'] and len(results['ids'][0]) > 0:
                retrieved_ids = results['ids'][0]
                retrieved_labels = [train_labels_map.get(id, -1) for id in retrieved_ids]
                all_results.append(retrieved_labels)
            else:
                all_results.append([])
            
            test_labels.append(int(label))
            tested_count += 1
    
    retrieval_time = time() - start_time
    
    # Tính metrics
    metrics_data = {}
    
    for k in K_VALUES:
        map_scores = []
        recall_scores = []
        hit_scores = []
        
        for retrieved_labels, gt_label in zip(all_results, test_labels):
            if len(retrieved_labels) == 0:
                map_scores.append(0.0)
                recall_scores.append(0.0)
                hit_scores.append(0.0)
                continue
            
            # Top-k results
            top_k_labels = retrieved_labels[:k]
            
            # Tính metrics
            ap = average_precision(top_k_labels, [gt_label], k)
            relevant_indices = [i for i, label in enumerate(train_labels_map.values()) if label == gt_label]
            rec = recall(list(range(len(top_k_labels))), relevant_indices, k)
            hr = hit_rate(top_k_labels, [gt_label], k)
            
            map_scores.append(ap)
            recall_scores.append(rec)
            hit_scores.append(hr)
        
        # Lưu metrics
        metrics_data[f'mAP@{k}'] = np.mean(map_scores)
        metrics_data[f'Recall@{k}'] = np.mean(recall_scores)
        metrics_data[f'HitRate@{k}'] = np.mean(hit_scores)
        
        print(f"   k={k}: mAP={np.mean(map_scores):.4f}, Recall={np.mean(recall_scores):.4f}, HR={np.mean(hit_scores):.4f}")
    
    # Tính average mAP
    avg_map = np.mean([metrics_data[f'mAP@{k}'] for k in K_VALUES])
    metrics_data['avg_mAP'] = avg_map
    
    # Thêm thông tin cấu hình
    metrics_data.update({
        'n_bin': n_bin,
        'h_type': h_type,
        'metric': metric,
        'config_name': f"{n_bin}bin_{h_type}_{metric}",
        'retrieval_time': retrieval_time,
        'tested_samples': tested_count,
        'train_samples': len(train_labels_map)
    })
    
    return metrics_data

print("📋 Hàm evaluation đã sẵn sàng!")


📋 Hàm evaluation đã sẵn sàng!


In [17]:
# CHẠY EVALUATION CHO TẤT CẢ CẤU HÌNH
print(f"🚀 Bắt đầu evaluation cho {len(CONFIGS)} cấu hình...")
print(f"📊 Sẽ test trên {TEST_SIZE} samples")
print(f"🎯 Metrics: mAP@{K_VALUES}")

evaluation_results = []

for i, (n_bin, h_type, metric) in enumerate(CONFIGS, 1):
    print(f"\n{'='*60}")
    print(f"Evaluation {i}/{len(CONFIGS)}: {n_bin}bin_{h_type}_{metric}")
    print(f"{'='*60}")
    
    try:
        result = evaluate_configuration(n_bin, h_type, metric)
        if result:
            evaluation_results.append(result)
            print(f"✅ Hoàn thành: avg_mAP = {result['avg_mAP']:.4f}")
        else:
            print(f"❌ Thất bại: {n_bin}bin_{h_type}_{metric}")
            
    except Exception as e:
        print(f"❌ Lỗi khi evaluate {n_bin}bin_{h_type}_{metric}: {e}")
        import traceback
        traceback.print_exc()

print(f"\n🎉 Hoàn thành evaluation cho {len(evaluation_results)} cấu hình!")


🚀 Bắt đầu evaluation cho 8 cấu hình...
📊 Sẽ test trên 5956 samples
🎯 Metrics: mAP@[1, 5, 10]

Evaluation 1/8: 4bin_global_cosine
📊 Found 1632 training vectors


Testing: 100%|██████████| 187/187 [13:02<00:00,  4.18s/it]


   k=1: mAP=0.0155, Recall=0.0004, HR=0.0311
   k=5: mAP=0.0276, Recall=0.0034, HR=0.0229
   k=10: mAP=0.0306, Recall=0.0063, HR=0.0199
✅ Hoàn thành: avg_mAP = 0.0246

Evaluation 2/8: 4bin_global_l2
📊 Found 23824 training vectors


Testing: 100%|██████████| 187/187 [16:06<00:00,  5.17s/it]


   k=1: mAP=0.0314, Recall=0.0000, HR=0.0628
   k=5: mAP=0.0492, Recall=0.0002, HR=0.0417
   k=10: mAP=0.0536, Recall=0.0004, HR=0.0362
✅ Hoàn thành: avg_mAP = 0.0447

Evaluation 3/8: 4bin_region_cosine
📊 Found 23824 training vectors


...

   k=1: mAP=0.0343, Recall=0.0000, HR=0.0687
   k=5: mAP=0.0544, Recall=0.0002, HR=0.0478
   k=10: mAP=0.0587, Recall=0.0004, HR=0.0408
✅ Hoàn thành: avg_mAP = 0.0491

Evaluation 7/8: 8bin_region_cosine
📊 Found 23824 training vectors


Testing: 100%|██████████| 187/187 [15:41<00:00,  5.04s/it]


   k=1: mAP=0.0410, Recall=0.0000, HR=0.0819
   k=5: mAP=0.0662, Recall=0.0002, HR=0.0631
   k=10: mAP=0.0711, Recall=0.0004, HR=0.0562
✅ Hoàn thành: avg_mAP = 0.0594

Evaluation 8/8: 8bin_region_l2
📊 Found 23824 training vectors


Testing: 100%|██████████| 187/187 [15:41<00:00,  5.04s/it]


   k=1: mAP=0.0427, Recall=0.0000, HR=0.0855
   k=5: mAP=0.0682, Recall=0.0002, HR=0.0645
   k=10: mAP=0.0727, Recall=0.0004, HR=0.0566
✅ Hoàn thành: avg_mAP = 0.0612

🎉 Hoàn thành evaluation cho 8 cấu hình!


In [18]:
# PHÂN TÍCH KẾT QUẢ
if evaluation_results:
    print(f"\n{'='*80}")
    print("📊 KẾT QUẢ ĐÁNH GIÁ TOÀN BỘ - DATASET 80/20")
    print(f"{'='*80}")
    
    # Sắp xếp theo avg_mAP
    sorted_results = sorted(evaluation_results, key=lambda x: x['avg_mAP'], reverse=True)
    
    # Hiển thị ranking
    print(f"\n🏆 XẾP HẠNG THEO AVERAGE mAP")
    print("-" * 85)
    print(f"{'Rank':<4} {'Configuration':<20} {'Avg mAP':<10} {'mAP@1':<8} {'mAP@5':<8} {'mAP@10':<8} {'Time(s)':<8}")
    print("-" * 85)
    
    for i, result in enumerate(sorted_results, 1):
        print(f"{i:<4} {result['config_name']:<20} {result['avg_mAP']:<10.4f} "
              f"{result['mAP@1']:<8.4f} {result['mAP@5']:<8.4f} {result['mAP@10']:<8.4f} "
              f"{result['retrieval_time']:<8.1f}")
    
    # Cấu hình tốt nhất
    best = sorted_results[0]
    print(f"\n🥇 CẤU HÌNH TỐT NHẤT: {best['config_name']}")
    print(f"   Average mAP: {best['avg_mAP']:.4f}")
    print(f"   Training samples: {best['train_samples']}")
    print(f"   Test samples: {best['tested_samples']}")
    print(f"   Retrieval time: {best['retrieval_time']:.2f}s")
    for k in K_VALUES:
        print(f"   mAP@{k}: {best[f'mAP@{k}']:.4f}")
    
    # Lưu kết quả
    df = pd.DataFrame(sorted_results)
    csv_path = 'out/evaluation_results_chromadb.csv'
    df.to_csv(csv_path, index=False)
    print(f"\n💾 Kết quả chi tiết lưu tại: {csv_path}")
    
    # Lưu cấu hình tốt nhất
    best_config = {
        'config_name': best['config_name'],
        'n_bin': best['n_bin'],
        'h_type': best['h_type'],
        'metric': best['metric'],
        'avg_mAP': best['avg_mAP'],
        'train_samples': best['train_samples'],
        'test_samples': best['tested_samples'],
        **{f'mAP@{k}': best[f'mAP@{k}'] for k in K_VALUES}
    }
    
    with open('out/best_config_chromadb.json', 'w') as f:
        json.dump(best_config, f, indent=2)
    print("📋 Cấu hình tốt nhất lưu tại: out/best_config_chromadb.json")
    
    # Top 3 summary
    print(f"\n🏅 TOP 3 CẤU HÌNH:")
    emojis = ["🥇", "🥈", "🥉"]
    for i, result in enumerate(sorted_results[:3]):
        emoji = emojis[i] if i < len(emojis) else "🏅"
        print(f"{emoji} {result['config_name']}: avg_mAP={result['avg_mAP']:.4f} "
              f"(n_bin={result['n_bin']}, h_type={result['h_type']}, metric={result['metric']})")
    
    print(f"\n💾 ChromaDB Storage: {CHROMA_DIR}")
    print("🔍 Vector collections được lưu trữ để tái sử dụng")
    print("♻️  Lần chạy tiếp theo sẽ nhanh hơn nhờ cache")
    print("\n🎉 Hoàn thành toàn bộ quá trình evaluation!")
    
else:
    print("❌ Không có kết quả evaluation nào!")
    print("Vui lòng đảm bảo đã chạy indexing thành công trước.")

# GPU memory cleanup
if device.type == "cuda":
    torch.cuda.empty_cache()
    print("🧹 GPU memory cleaned up")



📊 KẾT QUẢ ĐÁNH GIÁ TOÀN BỘ - DATASET 80/20

🏆 XẾP HẠNG THEO AVERAGE mAP
-------------------------------------------------------------------------------------
Rank Configuration        Avg mAP    mAP@1    mAP@5    mAP@10   Time(s) 
-------------------------------------------------------------------------------------
1    4bin_region_cosine   0.0673     0.0468   0.0751   0.0799   1087.6  
2    4bin_region_l2       0.0662     0.0464   0.0734   0.0789   973.1   
3    8bin_region_l2       0.0612     0.0427   0.0682   0.0727   941.8   
4    8bin_region_cosine   0.0594     0.0410   0.0662   0.0711   941.9   
5    8bin_global_l2       0.0491     0.0343   0.0544   0.0587   1202.2  
6    8bin_global_cosine   0.0487     0.0339   0.0539   0.0583   971.9   
7    4bin_global_l2       0.0447     0.0314   0.0492   0.0536   966.5   
8    4bin_global_cosine   0.0246     0.0155   0.0276   0.0306   782.3   

🥇 CẤU HÌNH TỐT NHẤT: 4bin_region_cosine
   Average mAP: 0.0673
   Training samples: 23824
   Test